In [8]:
import shapefile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch

ModuleNotFoundError: No module named 'Polygon3'

In [ ]:
# zipcode shape data from http://gis-mclio.opendata.arcgis.com/
shp = shapefile.Reader("./shape/mke_zipcode/Zip_Code_Tabulation_Areas_ZCTA")
# EMS calls for service from MKE Open Data Portal
ems = pd.read_csv('./data/mfdems.csv')

In [ ]:
def plot_df_on_mke_zip_map(df):
    fig = plt.figure(figsize=(12,10))
    ax = plt.axes()
    ax.set_aspect('equal')

    # start bounds with MKE lat/long coordinates
    [left, bottom, right, top] = [-87.9063889,43.0388889,-87.9063889,43.0388889]

    for i, shape in enumerate(shp.shapes()):
        # define polygon fill color (facecolor) RGB values:
        R = df.pct[shape.record(i)[1]] / max(df.pct) # percentage value for alderman district
        G = 0.1
        B = 0.1
        nparts = len(shape.parts) # total parts
        if nparts == 1:
            polygon = Polygon(shape.points)
            patch = PolygonPatch(polygon, facecolor=[0,0,0.5], alpha=0.7, edgecolor=[0,0,0], zorder=2)
            ax.add_patch(patch)
        else: # loop over parts of each shape, plot separately
            for ip in range(nparts): # loop over parts, plot separately
                i0=shape.parts[ip]
                if ip < nparts-1:
                    i1 = shape.parts[ip+1]-1
                else:
                    i1 = len(shape.points)
                polygon = Polygon(shape.points[i0:i1+1])
                patch = PolygonPatch(polygon, facecolor=[0,0,0.5], alpha=0.7, zorder=2)
                ax.add_patch(patch)
        # bounds
        l = shape.bbox[0]
        if l < left: left = l
        b = shape.bbox[1]
        if b < bottom: bottom = b
        r = shape.bbox[2]
        if r > right: right = r
        t = shape.bbox[3]
        if t > top: top = t
    # use bbox (bounding box) to set plot limits
    plt.xlim(left,right)
    plt.ylim(bottom,top)
    plt.axis('off')
    fig.savefig('./images/EMS - ' + df.columns[0] + ' - 2018_zipcode.png')

In [ ]:
# MKE EMS calls - total
total = ems[['ZIP Code','Final Call for Service Type']].groupby(['ZIP Code'], as_index=True).agg(['count'])['Final Call for Service Type']
total.columns = ['Total']
total['pct'] = total['Total'] / sum(total['Total'])
plot_df_on_mke_zip_map(total)